In [3]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

# Load the dataset
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Normalize the data
x_train = x_train / 255.0
x_test = x_test / 255.0

# Reshape for CNN input
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

# Define the model
def create_model(optimizer='adam', filter_size=5, reg_strength=0.001):
    model = keras.Sequential([
        layers.Conv2D(32, (filter_size, filter_size), activation='relu', input_shape=(28, 28, 1), kernel_regularizer=regularizers.l2(reg_strength)),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(64, (filter_size, filter_size), activation='relu', kernel_regularizer=regularizers.l2(reg_strength)),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(reg_strength)),
        layers.BatchNormalization(),
        layers.GlobalAveragePooling2D(),

        layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(reg_strength)),
        layers.Dropout(0.4),
        layers.Dense(10, activation='softmax')
    ])

    model.compile(optimizer=optimizer,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Parameters
optimizer_choice = 'adam'
filter_size_choice = 5
reg_strength_choice = 0.001
batch_size_choice = 64
epochs_choice = 15

# Callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=3, min_lr=1e-5)
early_stopping = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_loss', mode='min')

# Build and train model
print(f"\nTraining Model: filter_size={filter_size_choice}, reg_strength={reg_strength_choice}, "
      f"batch_size={batch_size_choice}, optimizer='{optimizer_choice}'\n")

model = create_model(optimizer=optimizer_choice,
                     filter_size=filter_size_choice,
                     reg_strength=reg_strength_choice)

history = model.fit(
    x_train, y_train,
    epochs=epochs_choice,
    batch_size=batch_size_choice,
    validation_split=0.2,
    verbose=1,  # Shows output for each epoch
    callbacks=[reduce_lr, early_stopping, checkpoint]
)

# Evaluate on test data
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)

# Final Summary
print("\nFinal Training Summary:")
print(f"Epochs Completed: {len(history.history['loss'])}")
print(f"Optimizer        : {optimizer_choice}")
print(f"Filter Size      : {filter_size_choice}")
print(f"Reg Strength     : {reg_strength_choice}")
print(f"Batch Size       : {batch_size_choice}")
print(f"Test Accuracy    : {test_acc:.4f}")



Training Model: filter_size=5, reg_strength=0.001, batch_size=64, optimizer='adam'

Epoch 1/15
750/750 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.7730 - loss: 0.8959 - val_accuracy: 0.8509 - val_loss: 0.5985 - learning_rate: 0.0010
Epoch 2/15
750/750 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8771 - loss: 0.5153 - val_accuracy: 0.8461 - val_loss: 0.5385 - learning_rate: 0.0010
Epoch 3/15
750/750 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.8916 - loss: 0.4329 - val_accuracy: 0.8859 - val_loss: 0.4281 - learning_rate: 0.0010
Epoch 4/15
750/750 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.8994 - loss: 0.3924 - val_accuracy: 0.8636 - val_loss: 0.5007 - learning_rate: 0.0010
Epoch 5/15
750/750 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.9026 - loss: 0.3802 - val_accuracy: 0.8838 - val_loss: 0.4363 - learning_rate: 0.0010
Epoch 6/15
750/750 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9082 - loss: 0.3690 - val_accuracy: 0.8942 - val_loss: 0.4047 - learning_rate: 0.0010
Ep